# CNN Filter and Feature Visualization 

This notebook outlines various methods for looking at filters and feature maps in CNN models. I think visually, so this is really meant to help me understand what is going on under the hood. I hope you find it useful as well. 

**Note:** I've included a `/helper_functions.py` file in the `/src` directory.

In [246]:
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
from PIL import Image
from PIL import ImageFile
import matplotlib.pyplot as plt
from helper_functions import plot_filter

ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'

## Load the Model
Here we will load a pretrained model from Pytorch. We will be using the VGG16 model in evaluation mode. Let's print the model to take a look at the various layers. The model has three children:
* features
* avgpool
* classifier

It is the features that we are interested in extracting weights from. Essentially, these weights in the Conv2d layers make up the filters that will pass over each image during a forward pass through the network. Let's take a look at the model and each filter. 

In [138]:
model = models.vgg16(pretrained=True)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [199]:
model_weights, conv_layers = [], []
features = model.features
conv_counter = 0

for i in range(len(features)):
    if type(features[i]) == nn.Conv2d:
        conv_counter += 1
        model_weights.append(features[i].weight)
        conv_layers.append(features[i])

## Convolution Layers

I'll print out each convolutional layer to take a look at the depth and tensor size.  

In [189]:
for i, (weights, layers) in enumerate(zip(model_weights, conv_layers)):
    print('Conv layer {}: {} --> Shape: {}'.format(i+1, layers, weights.shape))
          

Conv layer 1: Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([64, 3, 3, 3])
Conv layer 2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([64, 64, 3, 3])
Conv layer 3: Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([128, 64, 3, 3])
Conv layer 4: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([128, 128, 3, 3])
Conv layer 5: Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([256, 128, 3, 3])
Conv layer 6: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([256, 256, 3, 3])
Conv layer 7: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([256, 256, 3, 3])
Conv layer 8: Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) --> Shape: torch.Size([512, 256, 3, 3])
Conv layer 9: Conv2d(512, 512, kerne

## Filters

Looking at the first filter in Conv layer 1 (`model_weights[idx of conv layer][idx of the filter]`) we can see that it has a `kernel_size = (3, 3)`.

In [214]:
plt.figure(figsize=[10, 10])
model_weights[0][0]

tensor([[[-0.5537,  0.1427,  0.5290],
         [-0.5831,  0.3566,  0.7657],
         [-0.6902, -0.0480,  0.4841]],

        [[ 0.1755,  0.0099, -0.0814],
         [ 0.0441, -0.0703, -0.2604],
         [ 0.1324, -0.1728, -0.1323]],

        [[ 0.3130, -0.1659, -0.4275],
         [ 0.4752, -0.0827, -0.4870],
         [ 0.6320,  0.0193, -0.2775]]], grad_fn=<SelectBackward>)

<Figure size 720x720 with 0 Axes>

In [228]:
print(features[0].out_channels)
out_channels = features[5].out_channels
print(int(np.sqrt(out_channels)))

64
11


In [247]:
plot_filter(model_weights, 1, 6, 8, 8)

TypeError: plot_filter() takes from 0 to 4 positional arguments but 5 were given